In [ ]:
import arcpy
import os
import glob
from tqdm import tqdm

# Set environment settings
arcpy.env.overwriteOutput = True  # Allow overwriting existing files
arcpy.env.workspace = r"F:\temp_arcgispro"  # Set the workspace for temporary files

# Define the lists
shapefile_list = glob.glob(r'H:\LSM_DATA\masked_layers\vectors\morpho\*.shp')  # List A: Shapefiles
raster_list = glob.glob(r'H:\LSM_DATA\masked_layers\rasters_wgs\slope\*.tif')  # List B: Raster files
final_directory = r"H:\LSM_DATA\masked_layers\rasters_wgs\morpho"  # Directory to save final outputs

# Ensure final directory exists
if not os.path.exists(final_directory):
    os.makedirs(final_directory)  # Create the directory if it doesn't exist

# Function to convert polygon shapefile to raster using 'OBJECTID'
def convert_polygon_to_raster(shapefile, field, output_raster, cell_size):
    """
    Converts a polygon shapefile to a raster using a specified field.

    Parameters:
        shapefile (str): Path to the input shapefile.
        field (str): Field used for raster values (e.g., 'OBJECTID').
        output_raster (str): Path for the output raster file.
        cell_size (float): Size of the raster cells.
    """
    arcpy.conversion.PolygonToRaster(shapefile, field, output_raster, "CELL_CENTER", "", cell_size)

# Function to project raster to WGS 1984
def project_to_wgs84(input_raster, output_raster):
    """
    Projects a raster to the WGS 1984 spatial reference system.

    Parameters:
        input_raster (str): Path to the input raster.
        output_raster (str): Path for the output projected raster.
    """
    wgs84 = arcpy.SpatialReference(4326)  # Define the WGS 1984 spatial reference
    arcpy.management.ProjectRaster(input_raster, output_raster, wgs84)

# Main processing loop
for shapefile in tqdm(shapefile_list, desc="Processing shapefiles"):
    shapefile_name = os.path.basename(shapefile).replace(".shp", "")  # Extract the name without extension

    # Define the temporary and final raster paths
    temp_raster = os.path.join(arcpy.env.workspace, f"{shapefile_name}_temp_raster.tif")
    clipped_raster = os.path.join(arcpy.env.workspace, f"{shapefile_name}_clipped.tif")
    final_raster = os.path.join(final_directory, f"{shapefile_name}.tif")  # Final output path

    # Find the corresponding raster from list B
    target_raster = None
    for raster in raster_list:
        if os.path.basename(raster).replace(".tif", "") == shapefile_name:  # Match raster name with shapefile
            target_raster = raster
            break  # Exit loop once a match is found

    if target_raster:
        # Get the extent and cell size from the target raster
        target_desc = arcpy.Describe(target_raster)  # Describe raster properties
        target_extent = target_desc.extent  # Get the extent of the raster
        target_cell_size = target_desc.meanCellHeight  # Get cell size (assuming square cells)

        # Convert the polygon shapefile to raster based on 'OBJECTID' field
        convert_polygon_to_raster(shapefile, 'OBJECTID', temp_raster, target_cell_size)

        # Clip the raster to match the extent of the target raster
        arcpy.management.Clip(temp_raster,
                              f"{target_extent.XMin} {target_extent.YMin} {target_extent.XMax} {target_extent.YMax}",
                              clipped_raster,
                              in_template_dataset=target_raster,  # Clipping template
                              nodata_value="NONE",  # Set nodata_value to NONE
                              clipping_geometry="NONE",  # No clipping by geometry
                              maintain_clipping_extent="MAINTAIN_EXTENT")  # Maintain the clipping extent

        # Project the raster to WGS 1984
        project_to_wgs84(clipped_raster, final_raster)

        # Clean up temporary rasters
        arcpy.management.Delete(temp_raster)  # Delete temporary raster
        arcpy.management.Delete(clipped_raster)  # Delete clipped raster
    else:
        print(f"No matching raster found for shapefile: {shapefile_name}")  # Print message if no match is found

print("Processing complete.")  # Indicate that processing is finished
